In [190]:
import pandas as pd
import os
import numpy as np
pd.set_option("display.max_columns", 500)

In [32]:
!ls combined

2008.csv 2010.csv 2012.csv 2014.csv 2016.csv 2018.csv
2009.csv 2011.csv 2013.csv 2015.csv 2017.csv 2019.csv


# Coalescing Yearly Files

In [144]:
# function to format columns into a standard style, coalesce names
def format_column_name(cols):
    cols = [col.lower() for col in cols]
    cols = [col.replace("\n", " ") for col in cols]
    cols = [col.replace(" ", "_") for col in cols]
    if "state" in cols: cols[cols.index("state")] = "plant_state"  
    if 'contract_exp_date' in cols: cols[cols.index('contract_exp_date')] =  'contract_expiration_date'  
    if 'respondent_frequency' in cols: cols[cols.index('respondent_frequency')] =  'reporting_frequency'  
    return cols

In [145]:
# loop through to get a common set of fields across all documents
common_fields = set()
for file in os.listdir("combined"):
    if ".csv" in file:
        df = pd.read_csv("combined/" + file, nrows=1)
        cols = format_column_name(df.columns)
        if len(common_fields) > 0:
            common_fields = common_fields.intersection(set(cols))
            unused = common_fields.union(set(cols))
        else: common_fields = set(cols)
common_fields = list(common_fields)
common_fields

['reporting_frequency',
 'average_heat_content',
 'plant_id',
 'coalmine_type',
 'coalmine_county',
 'operator_id',
 'coalmine_name',
 'plant_name',
 'fuel_cost',
 'average_ash_content',
 'regulated',
 'secondary_transportation_mode',
 'coalmine_state',
 'coalmine_msha_id',
 'quantity',
 'operator_name',
 'contract_expiration_date',
 'energy_source',
 'supplier',
 'year',
 'plant_state',
 'average_sulfur_content',
 'primary_transportation_mode',
 'fuel_group',
 'month']

In [146]:
# list all fields to check for typos, variations that havent been coalesced
all_fields = []
for file in os.listdir("combined"):
    if ".csv" in file:
        df = pd.read_csv("combined/" + file, nrows=1)
        cols = format_column_name(df.columns)
        all_fields = all_fields + cols
sorted(all_fields)

['average_ash_content',
 'average_ash_content',
 'average_ash_content',
 'average_ash_content',
 'average_ash_content',
 'average_ash_content',
 'average_ash_content',
 'average_ash_content',
 'average_ash_content',
 'average_ash_content',
 'average_ash_content',
 'average_ash_content',
 'average_heat_content',
 'average_heat_content',
 'average_heat_content',
 'average_heat_content',
 'average_heat_content',
 'average_heat_content',
 'average_heat_content',
 'average_heat_content',
 'average_heat_content',
 'average_heat_content',
 'average_heat_content',
 'average_heat_content',
 'average_mercury_content',
 'average_mercury_content',
 'average_mercury_content',
 'average_mercury_content',
 'average_mercury_content',
 'average_mercury_content',
 'average_mercury_content',
 'average_sulfur_content',
 'average_sulfur_content',
 'average_sulfur_content',
 'average_sulfur_content',
 'average_sulfur_content',
 'average_sulfur_content',
 'average_sulfur_content',
 'average_sulfur_content',


In [149]:
# get a list of all unique fields, remove unnamed cols
all_fields = list(set(all_fields))
all_fields = [col for col in all_fields if "unnamed" not in col]
all_fields

['reporting_frequency',
 'average_heat_content',
 'purchase_type',
 'plant_id',
 'coalmine_type',
 'coalmine_county',
 'operator_id',
 'coalmine_name',
 'plant_name',
 'natural_gas_supply_contract_type',
 'fuel_cost',
 'average_ash_content',
 'regulated',
 'contract_type',
 'secondary_transportation_mode',
 'chlorine_content',
 'coalmine_state',
 'coalmine_msha_id',
 'average_mercury_content',
 'quantity',
 'operator_name',
 'contract_expiration_date',
 'energy_source',
 'supplier',
 'natural_gas_delivery_contract_type',
 'year',
 'plant_state',
 'average_sulfur_content',
 'primary_transportation_mode',
 'fuel_group',
 'natural_gas_transportation_service',
 'moisture_content',
 'month']

In [175]:
# Append all of the dataframes into a single DF
df = None
for file in os.listdir("combined"):
    if ".csv" in file:
        df_temp = pd.read_csv("combined/" + file)
        df_temp.columns = format_column_name(df_temp.columns)
        for col in all_fields:
            if col not in df_temp.columns:
                df_temp[col] = None

        if type(df) == pd.DataFrame:
            df = df.append(df_temp[all_fields], ignore_index=True, verify_integrity=True)
            
            # print out how many lines in each, just to keep a track of things
            print(file, len(df_temp))
        else:
            df = df_temp[all_fields]
            print(file, len(df_temp))
            

            

2008.csv 61849
2009.csv 60141
2019.csv 7972
2018.csv 34195
2015.csv 34068
2014.csv 37484
2016.csv 30751
2017.csv 20870
2013.csv 36715
2012.csv 39087
2010.csv 58993
2011.csv 58994


In [173]:
len(df)

481119

In [172]:
len(df.drop_duplicates())


477884

In [176]:
df = df.drop_duplicates()

In [191]:
df.head()

,reporting_frequency,average_heat_content,purchase_type,plant_id,coalmine_type,coalmine_county,operator_id,coalmine_name,plant_name,natural_gas_supply_contract_type,fuel_cost,average_ash_content,regulated,contract_type,secondary_transportation_mode,chlorine_content,coalmine_state,coalmine_msha_id,average_mercury_content,quantity,operator_name,contract_expiration_date,energy_source,supplier,natural_gas_delivery_contract_type,year,plant_state,average_sulfur_content,primary_transportation_mode,fuel_group,natural_gas_transportation_service,moisture_content,month
0,M,23.100,None,3,SU,IMP,195.0,MINA PRIBBENOW,Barry,None,213.5,5.4,REG,C,NaN,None,CL,NaN,None,"259,412",Alabama Power Co,408,BIT,INTEROCEAN COAL,None,2008,AL,0.49,RV,Coal,F,None,1
1,M,22.800,None,3,SU,IMP,195.0,MINA PRIBBENOW,Barry,None,211.5,5.7,REG,C,NaN,None,CL,NaN,None,"52,241",Alabama Power Co,408,BIT,INTEROCEAN COAL,None,2008,AL,0.48,RV,Coal,F,None,1
2,M,1.039,None,3,NaN,NaN,195.0,NaN,Barry,None,863.1,0.0,REG,C,NaN,None,NaN,NaN,None,"2,783,619",Alabama Power Co,NaN,NG,BAY GAS PIPELINE,None,2008,AL,0.00,PL,Natural Gas,F,None,1
3,A,24.610,None,7,SU,007,195.0,ALABAMA COAL,Gadsden,None,277.6,14.7,REG,C,NaN,None,AL,NaN,None,"25,397",Alabama Power Co,1215,BIT,ALABAMA COAL,None,2008,AL,1.69,TR,Coal,F,None,1
4,A,24.446,None,7,S,145,195.0,flat rock #2,Gadsden,None,338.1,15.5,REG,S,NaN,None,AL,NaN,None,764,Alabama Power Co,1108,BIT,D & E MINING,None,2008,AL,0.84,TR,Coal,F,None,1


# Formatting Column Data

In [228]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 378045 entries, 0 to 481118
Data columns (total 33 columns):
reporting_frequency                   378045 non-null object
average_heat_content                  378045 non-null float64
purchase_type                         136470 non-null object
plant_id                              378045 non-null int64
coalmine_type                         158373 non-null object
coalmine_county                       157339 non-null object
operator_id                           378042 non-null float64
coalmine_name                         156133 non-null object
plant_name                            378042 non-null object
natural_gas_supply_contract_type      33617 non-null object
fuel_cost                             312877 non-null float64
average_ash_content                   378045 non-null float64
regulated                             378045 non-null object
contract_type                         241521 non-null object
secondary_transportation_mode    

## Cleaning up errant characters

In [186]:
df["average_heat_content"] = df["average_heat_content"].apply(lambda x: float(str(x).replace(",", "")))
df["average_heat_content"] = df["average_heat_content"].astype(float)

In [189]:
df["average_sulfur_content"] = df["average_sulfur_content"].astype(float)

In [226]:
df["average_ash_content"] = df["average_ash_content"].astype(float)

In [227]:
df["quantity"] = df["quantity"].apply(lambda x: float(str(x).replace(",", "")))
df["quantity"] = df["quantity"].astype(int)

In [229]:
df["month"] = df["month"].astype(int)
df["year"] = df["year"].astype(int)

## Fuel Cost - Target Variable

In [214]:
def format_fuel_cost(x):
    if x == ".": x = ""
    x = str(x).replace(",", "")
    return x
    
df["fuel_cost"] = df["fuel_cost"].apply(format_fuel_cost)

In [216]:
len(df[df["fuel_cost"] == ""])/len(df)

# 20% of entries have no value. Since it will be the target, they need to be removed.

0.20891890082111977

In [217]:
df = df[df["fuel_cost"] != ""].copy()

In [223]:
df["fuel_cost"] = df["fuel_cost"].astype(float)

In [230]:
df.head()

,reporting_frequency,average_heat_content,purchase_type,plant_id,coalmine_type,coalmine_county,operator_id,coalmine_name,plant_name,natural_gas_supply_contract_type,fuel_cost,average_ash_content,regulated,contract_type,secondary_transportation_mode,chlorine_content,coalmine_state,coalmine_msha_id,average_mercury_content,quantity,operator_name,contract_expiration_date,energy_source,supplier,natural_gas_delivery_contract_type,year,plant_state,average_sulfur_content,primary_transportation_mode,fuel_group,natural_gas_transportation_service,moisture_content,month
0,M,23.100,None,3,SU,IMP,195.0,MINA PRIBBENOW,Barry,None,213.5,5.4,REG,C,NaN,None,CL,NaN,None,259412,Alabama Power Co,408,BIT,INTEROCEAN COAL,None,2008,AL,0.49,RV,Coal,F,None,1
1,M,22.800,None,3,SU,IMP,195.0,MINA PRIBBENOW,Barry,None,211.5,5.7,REG,C,NaN,None,CL,NaN,None,52241,Alabama Power Co,408,BIT,INTEROCEAN COAL,None,2008,AL,0.48,RV,Coal,F,None,1
2,M,1.039,None,3,NaN,NaN,195.0,NaN,Barry,None,863.1,0.0,REG,C,NaN,None,NaN,NaN,None,2783619,Alabama Power Co,NaN,NG,BAY GAS PIPELINE,None,2008,AL,0.00,PL,Natural Gas,F,None,1
3,A,24.610,None,7,SU,007,195.0,ALABAMA COAL,Gadsden,None,277.6,14.7,REG,C,NaN,None,AL,NaN,None,25397,Alabama Power Co,1215,BIT,ALABAMA COAL,None,2008,AL,1.69,TR,Coal,F,None,1
4,A,24.446,None,7,S,145,195.0,flat rock #2,Gadsden,None,338.1,15.5,REG,S,NaN,None,AL,NaN,None,764,Alabama Power Co,1108,BIT,D & E MINING,None,2008,AL,0.84,TR,Coal,F,None,1


In [231]:
df.fuel_group.value_counts()

Natural Gas       182923
Coal              158119
Petroleum          33668
Petroleum Coke      2076
Other Gas           1257
Name: fuel_group, dtype: int64

In [240]:
df["primary_transportation_mode"].value_counts()

PL    133053
RR    103196
TR     50549
RV     29639
TC      2263
WT      2112
TP       458
GL       244
tr         1
Name: primary_transportation_mode, dtype: int64

In [241]:
df["primary_transportation_mode"] = df["primary_transportation_mode"].apply(lambda x: str(x).upper())

In [244]:
def fix_nan(x):
    if x == "NAN":
        return None
    else:
        return x
df["primary_transportation_mode"] = df["primary_transportation_mode"].apply(fix_nan)

In [252]:
df.coalmine_type = df.coalmine_type.apply(lambda x: str(x).upper())
df.coalmine_type = df.coalmine_type.apply(fix_nan)

In [262]:
df[df["year"] == 2008].contract_type.value_counts()

S    31094
C    30245
N      342
Name: contract_type, dtype: int64

In [263]:
def n_to_nc(x):
    if x == "N":
        return "NC"
    else:
        return x
df.contract_type = df.contract_type.apply(n_to_nc)

In [325]:
def find_str(x):
    if x =="" or x == None or x == 'None':
        return np.nan
    else:
        return x

In [280]:
 
df["chlorine_content"] = df["chlorine_content"].apply(lambda x: x.replace("None", ""))
df["chlorine_content"] = df["chlorine_content"].apply(lambda x: x.replace(".", ""))

df["chlorine_content"] = df["chlorine_content"].apply(find_str)
df["chlorine_content"] = df["chlorine_content"].astype(float)

In [286]:
df.average_mercury_content = df.average_mercury_content.astype(float)

In [298]:
df.energy_source.value_counts()

NG     182923
BIT     96592
SUB     56893
DFO     29235
RFO      3102
WC       2444
LIG      2156
PC       2076
OG        927
KER       619
WO        574
BFG       190
JF        138
PG        122
SC         34
SGP        18
Name: energy_source, dtype: int64

In [294]:
df[df.energy_source == "151"]

,reporting_frequency,average_heat_content,purchase_type,plant_id,coalmine_type,coalmine_county,operator_id,coalmine_name,plant_name,natural_gas_supply_contract_type,fuel_cost,average_ash_content,regulated,contract_type,secondary_transportation_mode,chlorine_content,coalmine_state,coalmine_msha_id,average_mercury_content,quantity,operator_name,contract_expiration_date,energy_source,supplier,natural_gas_delivery_contract_type,year,plant_state,average_sulfur_content,primary_transportation_mode,fuel_group,natural_gas_transportation_service,moisture_content,month,temp
129329,M,25.162,C,8848,None,NaN,6455.0,NaN,Ceredo,NaN,205.6,8.2,REG,None,NaN,NaN,NaN,NaN,0.0,57783,"Duke Energy Florida, LLC",1219,151,ALLIANCE COAL,NaN,2019,WV,2.69,RV,NaN,None,.,4,NaN


In [297]:
df.loc[129329,"energy_source"] = np.nan

In [310]:
def natgas_trans(x):
    if x in ["F", "I", np.nan]:
        return x
    else:
        return np.nan
df.natural_gas_transportation_service = df.natural_gas_transportation_service.apply(natgas_trans)

In [319]:
df.moisture_content = df.moisture_content.apply(lambda x: str(x).replace(".", ""))

In [327]:
df.moisture_content = df.moisture_content.apply(find_str)

In [329]:
df.moisture_content = df.moisture_content.astype(float)

# Dealing with NAs

In [361]:
df.isna().sum()

reporting_frequency                        0
average_heat_content                       0
purchase_type                         241575
plant_id                                   0
coalmine_type                         219672
coalmine_county                       220706
operator_id                                0
coalmine_name                         221912
plant_name                                 0
natural_gas_supply_contract_type      344428
fuel_cost                              65168
average_ash_content                        0
regulated                                  0
contract_type                         136524
secondary_transportation_mode         352734
chlorine_content                      368961
coalmine_state                        219428
coalmine_msha_id                      240662
average_mercury_content               241562
quantity                                   0
operator_name                              0
contract_expiration_date               98308
energy_sou

In [ ]:
# Fill ID/name fields with unique ID

In [359]:
for i in df[df.plant_state.isna()].index:
    df.loc[i, "plant_state"] = "plant_" + str(i)
    df.loc[i, "operator_id"] = "operator_" + str(i)
    df.loc[i, "operator_name"] = "operator_" + str(i)
    df.loc[i, "plant_id"] = "plant_" + str(i)

In [364]:
df[df.fuel_group.isna()]

,average_ash_content,average_heat_content,average_mercury_content,average_sulfur_content,chlorine_content,coalmine_county,coalmine_msha_id,coalmine_name,coalmine_state,coalmine_type,contract_expiration_date,contract_type,energy_source,fuel_cost,fuel_group,moisture_content,month,natural_gas_delivery_contract_type,natural_gas_supply_contract_type,natural_gas_transportation_service,operator_id,operator_name,plant_id,plant_name,plant_state,primary_transportation_mode,purchase_type,quantity,regulated,reporting_frequency,secondary_transportation_mode,supplier,year
129329,8.2,25.162,0.0,2.69,NaN,NaN,NaN,NaN,NaN,None,1219,None,NaN,205.6,NaN,NaN,4,NaN,NaN,NaN,6455,"Duke Energy Florida, LLC",8848,Ceredo,WV,RV,C,57783,REG,M,NaN,ALLIANCE COAL,2019
228233,0.0,5.560,0.0,0.00,NaN,NaN,NaN,NaN,NaN,None,.,None,NaN,1953.7,NaN,NaN,10,None,None,NaN,5860,Empire District Electric Co,6223,Empire Energy Center,MO,TR,S,15176,REG,M,NaN,AVFUEL,2014


In [363]:
df = df.reindex(sorted(df.columns), axis=1)

In [365]:
df.to_csv("cleaned.csv")